# Analytical approaches

One might ask whether the diversity-expertise tradeoff (as modelled by the evidential sources model) can be studied analytically, using approaches from the voting literature. To investigate this, the notebook covers:
1. Lower bound in terms of number of sources and their mean reliability
2. The Cantelli lower bound (in terms of $\mu$ and $\sigma$)
3. Normal approximation. 

> Conclusion: None of these approaches seem useful for studying the diversity-expertise tradeoff in the considered parameter space.

In [1]:
import os
import math
import itertools

from tqdm.auto import tqdm
from scipy.stats import binom, norm
from typing import Tuple
from IPython.display import display

import pandas as pd
import numpy as np
import random as rd

from models.generate_teams import generate_expert_team, generate_diverse_team
from models.team import Team
from models.agent import Agent
from models.sources import Sources

# Lower bound in terms of number of sources and their mean reliability

### Implementation

In [2]:
def lower_bound_average(team: Team) -> float:
    """ Calculate a lower bound on the probability that the team makes the correct decision 
    (for the evidence-based mechanism) based on the average reliability of the sources they access."""
    sources = team.sources
    sources_accessed = set([s for a in team.members for s in a.heuristic])
    reliabilities_team_sources = [sources.reliabilities[s] for s in sources_accessed]
    n = len(reliabilities_team_sources)
    mean = sum(reliabilities_team_sources) / n
    threshold = math.ceil(n / 2)
    return binom.sf(threshold, n, mean)

sources = Sources(17, ("equi", 0.65, 0.2))
e_team = generate_expert_team(sources, 5, 9)
lower_bound_average(e_team)

0.729659098

In [3]:
def upper_bound_max(team: Team) -> float:
    """ Calculate an upper bound on the probability that the team makes the correct decision
    (for the evidence-based mechanism) based on the maximum reliability of the sources they access."""
    sources = team.sources
    sources_accessed = set([s for a in team.members for s in a.heuristic])
    reliabilities_team_sources = [sources.reliabilities[s] for s in sources_accessed]
    n = len(reliabilities_team_sources)
    rel_max = max(reliabilities_team_sources)
    threshold = math.ceil(n / 2)
    return binom.sf(threshold, n, rel_max)

sources = Sources(17, ("equi", 0.65, 0.2))
e_team = generate_expert_team(sources, 5, 9)
upper_bound_max(e_team)

0.8342742919921875

In [ ]:
def generate_lower_average_df() -> pd.DataFrame:
    """Generate a DataFrame comparing expert and diverse teams (for the evidence-based mechanisms)
    based on lower bound average reliability."""
    data = []
    n_sources_list = [13, 15, 17, 21]
    rel_mean_list = [0.55, 0.6, 0.65, 0.7, 0.75]
    rel_range_list = [0.15, 0.2, 0.25]
    team_size_list = [5, 7, 9]
    heuristic_size_list = [5, 7, 9, [5, 7, 9]]
    
    files = [
        file
        for file in os.listdir("data")
        if file.split("_")[0] == "simulation"
    ]
    for file in tqdm(files):
        df = pd.read_csv(f"data/{file}")
        for n_sources, rel_mean, rel_range, team_size, heuristic_size in itertools.product(
            n_sources_list, rel_mean_list, rel_range_list, team_size_list, heuristic_size_list
        ):        
            heuristic_str = heuristic_size if isinstance(heuristic_size, int) else "-".join(map(str, heuristic_size))                 
            if (
                heuristic_str in df.heuristic_size.values
                and team_size in df.team_size.values
                and rel_range in df.reliability_range.values
                and n_sources in df.n_sources.values
                and rel_mean in df.reliability_mean.values
            ):
                if "diverse" in df.team_type.values:
                    df_diverse = df[df["team_type"] == "diverse"]
                    d_team_accuracy = df_diverse["accuracy_evidence"].median()
                if "expert" in df.team_type.values:
                    df_expert = df[df["team_type"] == "expert"]
                    e_team_accuracy = df_expert["accuracy_evidence"].median()
        
                # Compute lower bound
                sources = Sources(n_sources, ("equi", rel_mean, rel_range))
                d_team = generate_diverse_team(
                    sources=sources, heuristic_size=heuristic_size, team_size=team_size) 
                d_lower = lower_bound_average(d_team)
                d_deviation = d_team_accuracy - d_lower
        
                DTE = True if d_team_accuracy > e_team_accuracy else False
                DTE_approx = True if d_lower > e_team_accuracy else False
        
                data.append([
                    n_sources,
                    rel_mean,
                    rel_range,
                    team_size, 
                    heuristic_str,
                    e_team_accuracy,
                    d_deviation,
                    d_lower,
                    d_team_accuracy,
                    DTE,
                    DTE_approx, 
                ])
    df = pd.DataFrame(data, columns=[
        "n_sources",
        "rel_mean",
        "rel_range",
        "team_size",
        "heuristic_size",
        "e_exact",
        "d_deviation",
        "d_lower",
        "d_exact",
        "DTE_exact",
        "DTE_approx",
    ])
    return df

### Results

Conclusion:
- The lower bound is not useful to approximate the diversity-expertise tradeoff, since the lower bound for the diverse team is always lower than the expert team’s accuracy, in the considered parameter space.
    - Compare columns `DTE_exact` and `DTE_approx` below.
    - `DTE_exact` shows the diversity-expertise tradeoff in the simulations. (`True` means that diversity trumped expertise.)
    - `DTE_approx` shows the approximated diversity-expertise tradeoff, using the lower bound in terms of number of sources and their mean reliability. 

In [5]:
df_lower_average = generate_lower_average_df()

  0%|          | 0/80 [00:00<?, ?it/s]

In [6]:
if df_lower_average["DTE_approx"].eq(False).all():
    print("All approximations indicate no DTE!")
else:
    print("Some approximations indicate DTE!")
print("="*40)
with pd.option_context('display.max_rows', None):
    display(df_lower_average)

All approximations indicate no DTE!


,n_sources,rel_mean,rel_range,team_size,heuristic_size,e_exact,d_deviation,d_lower,d_exact,DTE_exact,DTE_approx
0,13,0.55,0.20,9,5,0.698371,0.218031,0.426806,0.644837,False,False
1,13,0.60,0.20,9,5,0.799917,0.198637,0.574396,0.773033,False,False
2,13,0.65,0.20,9,5,0.880559,0.156791,0.715893,0.872684,False,False
3,13,0.70,0.20,9,5,0.937939,0.104941,0.834603,0.939544,True,False
4,13,0.75,0.20,9,5,0.973450,0.057278,0.919787,0.977065,True,False
5,17,0.55,0.20,9,5,0.733887,0.189435,0.474308,0.663743,False,False
6,17,0.60,0.20,9,5,0.828862,0.162454,0.640508,0.802962,False,False
7,17,0.65,0.20,9,5,0.901800,0.115337,0.787238,0.902575,True,False
8,17,0.70,0.20,9,5,0.951614,0.065845,0.895360,0.961205,True,False
9,17,0.75,0.20,9,5,0.980822,0.028684,0.959763,0.988447,True,False


# Cantelli lower bound

### Implementation

In [7]:
def joint_accuracy(agent1: Agent, agent2: Agent) -> float:
    """
    Compute the probability that two agents are BOTH correct.
    Assumes conditional independence given the true state.
    """
    sources = agent1.sources  # assuming both agents share the same Sources object
    rels_shared = [sources.reliabilities[s] for s in agent1.heuristic if s in agent2.heuristic]
    rels_unique_1 = [sources.reliabilities[s] for s in agent1.heuristic if s not in agent2.heuristic]
    rels_unique_2 = [sources.reliabilities[s] for s in agent2.heuristic if s not in agent1.heuristic]
    
    threshold_1, threshold_2 = (len(agent1.heuristic) + 1)//2, (len(agent2.heuristic) + 1)//2
    total = 0.0

    # Enumerate outcomes for shared sources first
    for shared_outcome in itertools.product([0,1], repeat=len(rels_shared)):
        p_shared = 1.0
        for source_rel, bit in zip(rels_shared, shared_outcome):
            p_shared *= source_rel if bit == 1 else (1 - source_rel)

        # agents probability of being correct given shared outcome
        def prob_agent_correct(rels_unique, shared_bits, threshold):
            total = 0.0
            for unique_outcome in itertools.product([0,1], repeat=len(rels_unique)):
                prob = 1.0
                for source_rel, bit in zip(rels_unique, unique_outcome):
                    prob *= source_rel if bit == 1 else (1 - source_rel)
                if sum(shared_bits) + sum(unique_outcome) >= threshold:
                    total += prob
            return total

        p_agent1_cond = prob_agent_correct(rels_unique_1, shared_outcome, threshold_1)
        p_agent2_cond = prob_agent_correct(rels_unique_2, shared_outcome, threshold_2)

        total += p_shared * p_agent1_cond * p_agent2_cond

    return total


def cantelli_bounds(team: Team) -> dict:
    """Given a team of agents, computes the agent accuracies and pairwise joint accuracies,
    then uses Cantelli's inequality to compute lower and upper bounds on the probability
    that the team majority is correct.
    
    Parameters
    ----------
        team: Team object
    
    Returns
    -------
        Lower and upper Cantelli bounds for group correctness.
    """    
    # Step 1: compute each agent's accuracy 
    agent_scores = [agent.score for agent in team.members]
    mu = sum(agent_scores)  # mean of the sum

    # Step 2: compute all pairwise joint accuracies p_ij
    cov_sum = 0.0
    for agent_i, agent_j in itertools.combinations(team.members, 2):
        p_ij = joint_accuracy(agent_i, agent_j)
        cov_sum += p_ij - agent_i.score*agent_j.score

    # Step 3: compute total variance
    var = sum(score_i*(1-score_i) for score_i in agent_scores) + 2 * cov_sum

    # Step 4: Cantelli bounds
    threshold = (team.size + 1) / 2
    lower = 1 - var / (var + (mu - (threshold - 1))**2)
    upper = var / (var + (threshold - mu)**2) if mu < threshold else 1.0

    return dict(mu=mu, var=var, lower=lower, upper=upper, q=agent_scores)

In [ ]:
def generate_cantelli_df() -> pd.DataFrame:
    """Generates a DataFrame comparing expert and diverse teams based on Cantelli bounds.
    
    Note: The lower bound is likely an overestimation: The lower bound for diverse teams 
    is based on a single diverse team, which means that the lower bound might be even lower 
    for other diverse teams."""
    data = []
    n_sources_list = [13, 15, 17, 21]
    rel_mean_list = [0.55, 0.6, 0.65, 0.7, 0.75]
    rel_range_list = [0.15, 0.2, 0.25]
    team_size_list = [5, 7, 9]
    heuristic_size_list = [5, 7, 9, [5, 7, 9]]
    
    files = [
        file
        for file in os.listdir("data")
        if file.split("_")[0] == "simulation"
    ]
    for file in tqdm(files):
        df = pd.read_csv(f"data/{file}")
        for n_sources, rel_mean, rel_range, team_size, heuristic_size in itertools.product(
            n_sources_list, rel_mean_list, rel_range_list, team_size_list, heuristic_size_list
        ):
            heuristic_str = heuristic_size if isinstance(heuristic_size, int) else "-".join(map(str, heuristic_size))
            if (
                heuristic_str in df.heuristic_size.values
                and team_size in df.team_size.values
                and rel_range in df.reliability_range.values
                and n_sources in df.n_sources.values
                and rel_mean in df.reliability_mean.values
            ):
                if "diverse" in df.team_type.values:
                    df_diverse = df[df["team_type"] == "diverse"]
                    d_team_accuracy = df_diverse["accuracy_opinion"].median()
                if "expert" in df.team_type.values:
                    df_expert = df[df["team_type"] == "expert"]
                    e_team_accuracy = df_expert["accuracy_opinion"].median()
        
                # Generate diverse team and compute Cantelli bounds
                sources = Sources(n_sources, ("equi", rel_mean, rel_range))
                team = generate_diverse_team(
                    sources=sources, heuristic_size=heuristic_size, team_size=team_size)
                result = cantelli_bounds(team)
                approx = result['lower']
                deviation = d_team_accuracy - approx
            
                data.append([
                    n_sources,
                    rel_mean,
                    rel_range,
                    team_size,
                    heuristic_str,
                    deviation,
                    approx,
                    d_team_accuracy,
                    e_team_accuracy,
                ])
    df_cantelli = pd.DataFrame(data, columns=[
        "n_sources",
        "rel_mean",
        "rel_range",
        "team_size",
        "heuristic_size",
        "d_deviation",
        "d_lower",
        "d_exact",
        "e_exact",
    ])
    df_cantelli.loc[:, "DTE_exact"] = df_cantelli["d_exact"] > df_cantelli["e_exact"]
    df_cantelli.loc[:, "DTE_approx"] = df_cantelli["d_lower"] > df_cantelli["e_exact"]
    return df_cantelli

### Results

Conclusion
- The Cantelli lower bound is not useful to approximate the diversity-expertise tradeoff, since the lower bound for the diverse team is always lower than the expert team’s accuracy, in the considered parameter space. 
    - Compare columns `DTE_exact` and `DTE_approx` below.
    - `DTE_exact` shows the diversity-expertise tradeoff in the simulations. (`True` means that diversity trumped expertise.)
    - `DTE_approx` shows the approximated diversity-expertise tradeoff, using the Cantelli lower bound. 

In [9]:
df_cantelli = generate_cantelli_df()

  0%|          | 0/80 [00:00<?, ?it/s]

In [10]:
if df_cantelli["DTE_approx"].eq(False).all():
    print("All approximations indicate no DTE.")
else:
    print("Some approximations indicate DTE.")
print("="*40)
with pd.option_context('display.max_rows', None):
    display(df_cantelli)

All approximations indicate no DTE.


,n_sources,rel_mean,rel_range,team_size,heuristic_size,d_deviation,d_lower,d_exact,e_exact,DTE_exact,DTE_approx
0,13,0.55,0.20,9,5,0.377381,0.259738,0.637119,0.717723,False,False
1,13,0.60,0.20,9,5,0.296773,0.463119,0.759892,0.803017,False,False
2,13,0.65,0.20,9,5,0.207816,0.650677,0.858494,0.873727,False,False
3,13,0.70,0.20,9,5,0.124420,0.803443,0.927863,0.927802,True,False
4,13,0.75,0.20,9,5,0.077759,0.891933,0.969692,0.965042,True,False
5,17,0.55,0.20,9,5,0.355982,0.296433,0.652415,0.735792,False,False
6,17,0.60,0.20,9,5,0.258152,0.526876,0.785029,0.818156,False,False
7,17,0.65,0.20,9,5,0.171306,0.713736,0.885042,0.885408,False,False
8,17,0.70,0.20,9,5,0.115849,0.832839,0.948688,0.935926,True,False
9,17,0.75,0.20,9,5,0.067635,0.914341,0.981976,0.969940,True,False


# Normal approximation

### Implementation

In [2]:
def joint_accuracy(agent1: Agent, agent2: Agent) -> float:
    """
    Computes the probability that two agents are BOTH correct.
    Assumes conditional independence given the true state.
    """
    sources = agent1.sources  # assuming both agents share the same Sources object
    rels_shared = [sources.reliabilities[s] for s in agent1.heuristic if s in agent2.heuristic]
    rels_unique_1 = [sources.reliabilities[s] for s in agent1.heuristic if s not in agent2.heuristic]
    rels_unique_2 = [sources.reliabilities[s] for s in agent2.heuristic if s not in agent1.heuristic]
    
    threshold_1, threshold_2 = (len(agent1.heuristic) + 1)//2, (len(agent2.heuristic) + 1)//2
    total = 0.0

    # Enumerate outcomes for shared sources first
    for shared_outcome in itertools.product([0,1], repeat=len(rels_shared)):
        p_shared = 1.0
        for source_rel, bit in zip(rels_shared, shared_outcome):
            p_shared *= source_rel if bit == 1 else (1 - source_rel)

        # agent probability of being correct given shared outcome
        def prob_agent_correct(rels_unique, shared_bits, threshold):
            total = 0.0
            for unique_outcome in itertools.product([0,1], repeat=len(rels_unique)):
                prob = 1.0
                for source_rel, bit in zip(rels_unique, unique_outcome):
                    prob *= source_rel if bit == 1 else (1 - source_rel)
                if sum(shared_bits) + sum(unique_outcome) >= threshold:
                    total += prob
            return total

        p_agent1_cond = prob_agent_correct(rels_unique_1, shared_outcome, threshold_1)
        p_agent2_cond = prob_agent_correct(rels_unique_2, shared_outcome, threshold_2)

        total += p_shared * p_agent1_cond * p_agent2_cond

    return total


def normal_approximation_group_accuracy(
    team: Team,
    continuity_correction: bool = True
) -> Tuple[float, float, float]:
    """Computes the normal approximation to the probability that the team majority is correct."""
    agent_scores = [agent.score for agent in team.members]
    mu = sum(agent_scores)
    
    # Compute all pairwise joint accuracies p_ij
    cov_sum = 0.0
    for agent_i, agent_j in itertools.combinations(team.members, 2):
        p_ij = joint_accuracy(agent_i, agent_j)
        cov_sum += p_ij - agent_i.score*agent_j.score

    # Compute total variance
    var = sum(score_i*(1-score_i) for score_i in agent_scores) + 2 * cov_sum
    sigma = math.sqrt(var)

    threshold = math.floor(team.size / 2) + 1
    # If degenerate variance
    if var <= 0:
        approx = 1.0 if mu >= threshold else 0.0
        return mu, var, approx

    # Normal approximation
    if continuity_correction:
        z = (threshold - 0.5 - mu) / sigma
    else:
        z = (threshold - mu) / sigma

    approx = 1 - norm.cdf(z)

    return mu, var, float(approx)


In [6]:
def generate_normal_approximation_df(team_type, n_diverse_samples: int = 5) -> pd.DataFrame:
    """Generates a DataFrame with normal approximations for the diverse and expert teams, 
    one for each simulation file in the data folder. 
    
    Note: Generates n_sample diverse teams and takes the worst diverse team as a lower bound."""
    data = []
    n_sources_list = [13, 15, 17, 21]
    rel_mean_list = [0.55, 0.6, 0.65, 0.7, 0.75]
    rel_range_list = [0.15, 0.2, 0.25]
    team_size_list = [5, 7, 9]
    heuristic_size_list = [5, 7, 9, [5, 7, 9]]
    
    # Retrieve exact accuracies from simulation data
    files = [
        file
        for file in os.listdir("data")
        if file.split("_")[0] == "simulation"
    ]
    for file in tqdm(files):
        df = pd.read_csv(f"data/{file}")
        for n_sources, rel_mean, rel_range, team_size, heuristic_size in itertools.product(
            n_sources_list, rel_mean_list, rel_range_list, team_size_list, heuristic_size_list
        ):
            heuristic_str = heuristic_size if isinstance(heuristic_size, int) else "-".join(map(str, heuristic_size))
            if (
                heuristic_str in df.heuristic_size.values
                and team_size in df.team_size.values
                and rel_range in df.reliability_range.values
                and n_sources in df.n_sources.values
                and rel_mean in df.reliability_mean.values
            ):
                if team_type in df.team_type.values:
                    df_team_type = df[df["team_type"] == team_type]
                    team_accuracy = df_team_type["accuracy_opinion"].median()
        
                # Generate team and compute normal approximation
                sources = Sources(n_sources, ("equi", rel_mean, rel_range))
                if team_type == "diverse":
                    results = []
                    for _ in range(n_diverse_samples):
                        team = generate_diverse_team(sources, heuristic_size, team_size)
                        mu, sigma2, approx = normal_approximation_group_accuracy(team)
                        results.append([mu, sigma2, approx])
                    approx_min = min([item[2] for item in results])
                    results_lower = [item for item in results if item[2] == approx_min]
                    lower_bound = rd.choice(results_lower)
                    mu, sigma2, approx = lower_bound                    
                    deviation = approx - team_accuracy
                elif team_type == "expert":
                    team = generate_expert_team(sources, heuristic_size, team_size)
                    mu, sigma2, approx = normal_approximation_group_accuracy(team)
                    deviation = approx - team_accuracy
                else:
                    raise ValueError("team_type must be 'diverse' or 'expert'")
                    
                
                data.append([
                    n_sources,
                    rel_mean,
                    rel_range,
                    team_size,
                    heuristic_str,
                    deviation,
                    approx,
                    mu, 
                    sigma2,
                    team_accuracy,
                ])
    df_normal = pd.DataFrame(data, columns=[
        "n_sources",
        "rel_mean",
        "rel_range",
        "team_size",
        "heuristic_size",
        "deviation",
        "approx",
        "mu",
        "sigma2",
        "exact",
    ])
    return df_normal

### Results

Conclusion:
- The normal approximation is not useful for approximating the diversity-expertise tradeoff: the expert team’s approximated accuracy (virtually) always exceeds the diverse team’s approximated accuracy in the considered parameter space---except for some cases where the reliability range was small (0.15). (Compare columns `DTE_exact` and `DTE_approx` below.)
    - `DTE_exact` shows the diversity-expertise tradeoff in the simulations. (`True` means that diversity trumped expertise.)
    - `DTE_approx` shows the approximated diversity-expertise tradeoff, using normal approximation. 

In [7]:
df_normal_e = generate_normal_approximation_df("expert")
df_normal_d = generate_normal_approximation_df("diverse")

df_normal = df_normal_e.copy()
df_normal.rename(columns={
    "deviation": "e_deviation",
    "approx": "e_approx",
    "mu": "e_mu",
    "sigma2": "e_sigma2",
    "exact": "e_exact",
}, inplace=True)
df_normal.loc[:, ["deviation", "approx", "mu", "sigma2", "exact"]] = df_normal_d[["deviation", "approx", "mu", "sigma2", "exact"]]
df_normal.rename(columns={
    "deviation": "d_deviation",
    "approx": "d_approx",
    "mu": "d_mu",
    "sigma2": "d_sigma2",
    "exact": "d_exact",
}, inplace=True)
df_normal.loc[:, "DTE_exact"] = df_normal["d_exact"] > df_normal["e_exact"]
df_normal.loc[:, "DTE_approx"] = df_normal["d_approx"] > df_normal["e_approx"]
# df_normal[["n_sources", "rel_mean", "e_approx", "e_exact", "d_approx", "d_exact", "DTE_exact", "DTE_approx"]]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [8]:
df_normal_sub = df_normal[["n_sources", "rel_mean", "rel_range", "team_size", "heuristic_size", "e_approx", "e_exact", "d_approx", "d_exact", "DTE_exact", "DTE_approx"]]
all_false = df_normal["DTE_approx"].eq(False).all()
if all_false:
    print("All approximations indicate no DTE.")
else:
    print("Some approximations indicate DTE.")
    n_false_total = len(df_normal[df_normal["DTE_exact"] == False])
    n_true_total = len(df_normal[df_normal["DTE_exact"] == True])
    df_eq = df_normal[df_normal["DTE_approx"] == df_normal["DTE_exact"]]
    n_false_correct = len(df_eq[df_eq["DTE_exact"] == False])
    n_true_correct = len(df_eq[df_eq["DTE_exact"] == True])
    print(f"--- Approximation correct false rate: {n_false_correct} / {n_false_total}")
    print(f"--- Approximation correct true rate: {n_true_correct} / {n_true_total}")
    if n_false_correct != n_false_total:
        display(df_normal_sub[df_normal["DTE_approx"] == False])
    if n_true_correct != n_true_total:
        display(df_normal_sub[df_normal["DTE_approx"] == True])

print("="*40)
print("All results")
print("="*40)

with pd.option_context('display.max_rows', None):
    display(df_normal_sub)

Some approximations indicate DTE.
--- Approximation correct false rate: 59 / 59
--- Approximation correct true rate: 3 / 21


,n_sources,rel_mean,rel_range,team_size,heuristic_size,e_approx,e_exact,d_approx,d_exact,DTE_exact,DTE_approx
37,17,0.65,0.15,9,5,0.891445,0.863428,0.899023,0.884236,True,True
38,17,0.70,0.15,9,5,0.964122,0.919733,0.970861,0.947846,True,True
39,17,0.75,0.15,9,5,0.996564,0.959339,0.997614,0.981555,True,True


All results


,n_sources,rel_mean,rel_range,team_size,heuristic_size,e_approx,e_exact,d_approx,d_exact,DTE_exact,DTE_approx
0,13,0.55,0.20,9,5,0.712611,0.717723,0.629980,0.637119,False,False
1,13,0.60,0.20,9,5,0.812087,0.803017,0.749635,0.759892,False,False
2,13,0.65,0.20,9,5,0.904629,0.873727,0.863250,0.858494,False,False
3,13,0.70,0.20,9,5,0.972467,0.927802,0.955143,0.927863,True,False
4,13,0.75,0.20,9,5,0.998250,0.965042,0.993387,0.969692,True,False
5,17,0.55,0.20,9,5,0.732942,0.735792,0.642409,0.652415,False,False
6,17,0.60,0.20,9,5,0.831585,0.818156,0.781795,0.785029,False,False
7,17,0.65,0.20,9,5,0.920609,0.885408,0.884511,0.885042,False,False
8,17,0.70,0.20,9,5,0.980748,0.935926,0.971217,0.948688,True,False
9,17,0.75,0.20,9,5,0.999267,0.969940,0.997124,0.981976,True,False
